In [56]:
import json, unidecode, re
from pathlib import Path

data = Path('data')

# Load the json file
with (data / 'animes.json').open() as f:
	animes = json.load(f)

animes[0]

{'mal_id': 52034,
 'url': 'https://myanimelist.net/anime/52034/Oshi_no_Ko',
 'images': {'jpg': {'image_url': 'https://cdn.myanimelist.net/images/anime/1812/134736.jpg',
   'small_image_url': 'https://cdn.myanimelist.net/images/anime/1812/134736t.jpg',
   'large_image_url': 'https://cdn.myanimelist.net/images/anime/1812/134736l.jpg'},
  'webp': {'image_url': 'https://cdn.myanimelist.net/images/anime/1812/134736.webp',
   'small_image_url': 'https://cdn.myanimelist.net/images/anime/1812/134736t.webp',
   'large_image_url': 'https://cdn.myanimelist.net/images/anime/1812/134736l.webp'}},
 'trailer': {'youtube_id': '1yXa8MAmocQ',
  'url': 'https://www.youtube.com/watch?v=1yXa8MAmocQ',
  'embed_url': 'https://www.youtube.com/embed/1yXa8MAmocQ?enablejsapi=1&wmode=opaque&autoplay=1',
  'images': {'image_url': 'https://img.youtube.com/vi/1yXa8MAmocQ/default.jpg',
   'small_image_url': 'https://img.youtube.com/vi/1yXa8MAmocQ/sddefault.jpg',
   'medium_image_url': 'https://img.youtube.com/vi/1yXa

# Aggregate animes into franchises

In [57]:
def sanitize(text: str):
	# Unidecode
	text = unidecode.unidecode(text)
	# Lowercase
	text = text.lower()
	# Remove special characters
	text = re.sub(r"[^\w\s]", "", text)
	# Remove multiple spaces
	text = re.sub(r"\s+", " ", text)
	# Remove leading and trailing spaces
	text = text.strip()
	return text

# Return the name of the franchise, none if not found
def get_franchise(a_title: str, f_title: str, auto: bool):
	words1 = a_title.split(" ")
	words2 = f_title.split(" ")
	min_w_len = min(len(words1), len(words2))

	# If franchise isn't auto, and we find a common string, it is a franchise
	match = re.search(f_title, a_title)
	if not auto and match:
		# print(f"Franchise match (manual)")
		return f_title

	common = []
	for i in range(min_w_len):
		if sanitize(words1[i]) == sanitize(words2[i]):
			common.append(words1[i])
		else:
			break
	franchise = " ".join(common)

	# If more than XX% of the characters of the shortest title are common, it is a franchise
	min_len = min(len(a_title), len(f_title))
	if len(franchise) / min_len > 0.8:
		# print(f"Franchise match (XX%): {len(franchise)}/{min_len}")
		return franchise

	# If the length of the common string is more than X characters, it is a franchise
	if len(franchise) > 15:
		# print(f"Franchise match (X characters)")
		return franchise

	return None

known_franchises = [
	"Evangelion",
	"Code Geass",
]

franchises = []
# Initialize franchises list with known franchises
for franchise in known_franchises:
	franchises.append({
		"animes": [],
		"title": franchise,
		"auto": False,
	})

# Build franchises list
for anime in animes:
	match = None
	index = None

	for i in range(len(franchises)):
		franchise = franchises[i]
		match = get_franchise(anime["title"], franchise["title"], franchise["auto"])
		if match:
			# print(f"Franchise of [{anime['title']}] and [{franchises[i]['title']}] is [{match}]")
			index = i
			break
	
	if not match:
		franchises.append({
			"animes": [anime],
			"title": anime["title"],
			"auto": True,
		})
		continue

	franchises[index]["animes"].append(anime)
	franchises[index]["title"] = match

print("Found " + str(len(franchises)) + " franchises")
for franchise in franchises:
	titles = [anime["title"] for anime in franchise["animes"]]
	print(f"- {franchise['title']} [{', '.join(titles)}]")

Found 60 franchises
- Evangelion [Evangelion: 1.0 You Are (Not) Alone, Evangelion: 2.0 You Can (Not) Advance, Evangelion: 3.0 You Can (Not) Redo, Evangelion: 3.0+1.0 Thrice Upon a Time, Neon Genesis Evangelion]
- Code Geass [Code Geass: Fukkatsu no Lelouch, Code Geass: Hangyaku no Lelouch, Code Geass: Hangyaku no Lelouch R2]
- "Oshi no Ko" ["Oshi no Ko"]
- 86 [86, 86 Part 2]
- Angel Beats! [Angel Beats!]
- Ansatsu Kyoushitsu [Ansatsu Kyoushitsu, Ansatsu Kyoushitsu 2nd Season]
- Berserk: Ougon Jidai-hen [Berserk: Ougon Jidai-hen I - Haou no Tamago, Berserk: Ougon Jidai-hen II - Doldrey Kouryaku, Berserk: Ougon Jidai-hen III - Kourin]
- Blue Lock [Blue Lock]
- Bocchi the Rock! [Bocchi the Rock!]
- Boku no Hero Academia [Boku no Hero Academia, Boku no Hero Academia 2nd Season, Boku no Hero Academia 3rd Season, Boku no Hero Academia 4th Season, Boku no Hero Academia 5th Season, Boku no Hero Academia 6th Season, Boku no Hero Academia the Movie 1: Futari no Hero, Boku no Hero Academia the Mo

# Compute franchise stats

In [58]:
def weighted_mean(animes, attr, total_episodes):
	if total_episodes > 0:  # Check to avoid division by zero
		return sum((anime[attr] if anime[attr] is not None else 0) * 
				   (anime["episodes"] if anime["episodes"] is not None else 0) 
				   for anime in animes) / total_episodes
	return None

def union(animes, attr):
	return list(set(name['name'] for anime in animes for name in anime[attr]))

franchises_clean = []
for franchise in franchises:
	animes = franchise['animes']
	clean = {}
	clean['title'] = franchise['title']
	clean['episodes'] = sum(anime["episodes"] if anime["episodes"] is not None else 0 for anime in animes)
	clean['score'] = weighted_mean(animes, "score", clean['episodes'])
	clean['my_score'] = weighted_mean(animes, "my_score", clean['episodes'])
	clean['genres'] = union(animes, "genres")
	clean['themes'] = union(animes, "themes")
	clean['demographics'] = union(animes, "demographics")
	clean['studios'] = union(animes, "studios")
	clean['licensors'] = union(animes, "licensors")
	clean['producers'] = union(animes, "producers")
	franchises_clean.append(clean)
	
franchises_clean[0]

{'title': 'Evangelion',
 'episodes': 30,
 'score': 8.321333333333333,
 'my_score': 8.033333333333333,
 'genres': ['Drama',
  'Suspense',
  'Avant Garde',
  'Award Winning',
  'Action',
  'Sci-Fi'],
 'themes': ['Psychological', 'Mecha'],
 'demographics': [],
 'studios': ['Tatsunoko Production', 'Gainax', 'Khara'],
 'licensors': ['Funimation', 'ADV Films', 'GKIDS'],
 'producers': ['TV Tokyo',
  'Nihon Ad Systems',
  'Kadokawa Shoten',
  'Audio Tanaka']}

In [59]:
# Save the franchises
with (data / 'franchises.json').open('w') as f:
	json.dump(franchises_clean, f)